In [7]:
# !pip install -q tensorflow

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import os
import shutil
import matplotlib.pyplot as plt

print("Versão do Tensorflow:")
print(tf.__version__)

Versão do Tensorflow:
2.20.0


# Classificação de Elementos Eletrônicos com TensorFlow e Keras - Tarefa 24

## Carregar dados

In [8]:
# Configurações
IMG_SIZE = (128, 256)
BATCH_SIZE = 32
SEED = 42

IMAGE_BASE_PATH = "../data/images"

# Carregar TODOS os dados
full_dataset = tf.keras.utils.image_dataset_from_directory(
    IMAGE_BASE_PATH,
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    shuffle=True
)

# Obter informações
class_names = full_dataset.class_names
num_classes = len(class_names)

print(f"Número de classes: {num_classes}")
print(f"Classes: {class_names}")

Found 10990 files belonging to 36 classes.
Número de classes: 36
Classes: ['Bypass-capacitor', 'Electrolytic-capacitor', 'Integrated-micro-circuit', 'LED', 'PNP-transistor', 'armature', 'attenuator', 'cartridge-fuse', 'clip-lead', 'electric-relay', 'filament', 'heat-sink', 'induction-coil', 'jumper-cable', 'junction-transistor', 'light-circuit', 'limiter-clipper', 'local-oscillator', 'memory-chip', 'microchip', 'microprocessor', 'multiplexer', 'omni-directional-antenna', 'potential-divider', 'potentiometer', 'pulse-generator', 'relay', 'rheostat', 'semi-conductor', 'semiconductor-diode', 'shunt', 'solenoid', 'stabilizer', 'step-down-transformer', 'step-up-transformer', 'transistor']


## Dividir dados em treino, validação e teste

In [ ]:
total_batches = tf.data.experimental.cardinality(full_dataset).numpy()
train_size = int(0.7 * total_batches)
val_size = int(0.15 * total_batches)
test_size = total_batches - train_size - val_size

# Dividir
train_dataset = full_dataset.take(train_size)
remaining = full_dataset.skip(train_size)
val_dataset = remaining.take(val_size)
test_dataset = remaining.skip(val_size)

print(f"Treino: {train_size} batches")
print(f"Validação: {val_size} batches")
print(f"Teste: {test_size} batches")

# Otimização com AUTOTUNE
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)

Treino: 240 batches
Validação: 51 batches
Teste: 53 batches
